# LSTM
## Long-Short Team Memory
##
### Tutorial based on: https://medium.com/datarisk-io/introdu%C3%A7%C3%A3o-%C3%A0s-redes-lstm-prevendo-valor-de-a%C3%A7%C3%B5es-na-bolsa-df270ca0cee5
##
## Application
<p>
The LSTM model can be applied for long temporal series since using other regression models like ARMA or ARIMA creates 1 polynomium for period used in the past to predict the future. Higher granularity increases the prediction model and don't deal with sazionalities. 

### Reading Recommendation
<p>
 Forecasting: Principles and Practice - Rob J. Hyndman and George Athanasopoulos

## PREDICT STOCK PRICES

In [28]:
# Import the required libraries 

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np 
import pandas as pd
import matplotlib as plt 


In [29]:
# Load data from Stocks and adjust the dataset for training

base = pd.read_csv('PETR4_Training.csv')

# Remove NA values from the dataset

base = base.dropna() 

# Select only the Opening Data and stora in an Array that will be used for training

base_treinamento = base.iloc[:,1:2].values

# Inspect the first 5 elementes of the array that will be used for training

base_treinamento[0:5]

array([[33.470001],
       [33.549999],
       [33.650002],
       [33.099998],
       [33.869999]])

In [30]:
# Scale the values of the dataset between 0 and 1

normalizador = MinMaxScaler(feature_range=(0,1))

base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)


# Inspect the first 5 elementes of the scaled training array

base_treinamento_normalizada[0:5]

array([[0.14523493],
       [0.1573375 ],
       [0.17246654],
       [0.08925869],
       [0.20574899]])

In [31]:
len(base_treinamento_normalizada)

90

In [69]:
# Prepare the data on the required structure to be trained by the LSTM model. We will define the temporal window to be observed

obs_window = 90

previsores=[]
preco_real=[]

# Vary i from obs_window value since this is the window size to observe

for i in range(0, 90):
    previsores.append(base_treinamento_normalizada[i-90:i,0])
    preco_real.append(base_treinamento_normalizada[i,0])
    
# Transform in the format to be used for training (this is required to make the model work)

previsores, preco_real = np.array(previsores), np.array(preco_real)
previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))


In [74]:
# Adicionamos também camadas de Dropout para prevenir ovefitting
regressor = Sequential()
regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores.shape[1], 1)))
regressor.add(Dropout(0.3))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.3))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))
regressor.add(Dense(units = 1, activation = 'linear'))
# Aqui definimos o otimizador, a função de perda e métricas de      
# avaliação do modelo durante o treinamento.
regressor.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',
                  metrics = ['mean_absolute_error'])

In [75]:
# Por fim, damos início ao treinamento. A taxa de aprendizado foi a padrão do otimizador,0.001.
regressor.fit(previsores, preco_real, epochs = 150, batch_size = 32)

Epoch 1/150


ValueError: Exception encountered when calling LSTM.call().

[1mslice index 0 of dimension 1 out of bounds. for '{{node sequential_2_1/lstm_8_1/strided_slice_1}} = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=5, ellipsis_mask=0, end_mask=5, new_axis_mask=0, shrink_axis_mask=2](data, sequential_2_1/lstm_8_1/strided_slice_1/stack, sequential_2_1/lstm_8_1/strided_slice_1/stack_1, sequential_2_1/lstm_8_1/strided_slice_1/stack_2)' with input shapes: [?,0,1], [3], [3], [3] and with computed input tensors: input[1] = <0 0 0>, input[2] = <0 1 0>, input[3] = <1 1 1>.[0m

Arguments received by LSTM.call():
  • sequences=tf.Tensor(shape=(None, 0, 1), dtype=float32)
  • initial_state=None
  • mask=None
  • training=True